In [1]:
import findspark
import re
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
from nltk.corpus import stopwords
import nltk 
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.linalg import Vector as MllibVector, Vectors as MLLibVectors

In [2]:
sc = pyspark.SparkContext()

In [3]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [4]:
#read in data
df = pd.read_csv("train.csv")
df = spark.createDataFrame(df)

In [5]:
df =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))

In [6]:
regexToken = RegexTokenizer(inputCol='plot',outputCol='tokens', pattern=" ")

In [7]:
df = regexToken.transform(df)

In [8]:
stop = stopwords.words('english')

In [9]:
stopWordsRemover = StopWordsRemover(inputCol = 'tokens',outputCol = 'cleaned').setStopWords(stop)

In [10]:
df = stopWordsRemover.transform(df)

In [11]:
counts = CountVectorizer(inputCol= 'cleaned',outputCol= 'features',minDF=10)

In [12]:
model = counts.fit(df)

In [13]:
df = model.transform(df)

In [14]:
 df =df.drop('plot','tokens')

In [15]:
labels = df.select('genre')

In [16]:
labels= labels.rdd.flatMap(lambda x:x)

In [17]:
feat = df.select('features')
feat = feat.rdd.flatMap(lambda x:x)

In [18]:
feats = feat.collect()

In [19]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [25]:
reg = LogisticRegression(regParam=0.01, weightCol='genre')
model = reg.fit(train)
pred = model.transform(testData)

("['World cinema', 'Drama']",
 SparseVector(24527, {126: 1.0, 180: 1.0, 499: 1.0, 536: 1.0, 576: 1.0, 1862: 1.0, 2079: 1.0, 3134: 1.0, 5240: 1.0, 8191: 1.0, 13228: 1.0, 21435: 1.0, 21886: 1.0}))